In [1]:
import pandas as pd
from Acesso import Login
from Query import Query

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)



In [2]:
querys={
    
    'Vendas':
    
    """
    
    SELECT * FROM netfeira.vw_targetestatistico
    
    """,
    
    'Produto':
    
    """
    
    SELECT * FROM netfeira.vw_produto
    
    """,
    
    'Cliente':
    
    """
    
    SELECT * FROM netfeira.vw_cliente
    
    
    """,
    
    'Vendedor':
    
    """
    
    SELECT * FROM netfeira.vw_vendedor
    
    """,
    
    'Supervisor':
    
    """
    
    SELECT * FROM netfeira.vw_supervisor
        
    """
    
    
}

In [3]:
tabelas=sql.CriarTabela(kwargs=querys)

In [4]:
vendas_df=pd.DataFrame()

vendas_df=tabelas['Vendas']

In [5]:
vendas_df.columns

Index(['Origem', 'Situação Entrega', 'ID Situação', 'Situação', 'Pedido',
       'NFe', 'Tipo de Pedido', 'Tipo de Operação', 'Tabelas',
       'Data de Emissão', 'Data de Faturamento', 'Data de Entrega',
       'ID Empresa', 'ID Cliente', 'ID Vendedor', 'SKU', 'Seq', 'Qtde',
       'Unid. VDA', 'Fator', 'Qtde. VDA', 'Valor VDA', 'Total Venda',
       'Total AV', 'MG CRP', 'Margem CUE', 'Margem CMP', 'Comsissão R$',
       'COFINS R$', 'PIS R$', 'ICMS R$', 'ICMS ST R$', 'IPI R$',
       'Peso Bruto KG', 'Peso Líquido KG', 'Total Geral'],
      dtype='object')

In [6]:
vendas_df=vendas_df.loc[(vendas_df['Tipo de Operação']!='OUTROS')&(vendas_df['ID Situação'].isin(['FA']))]

In [7]:
tabelas['Produto'].columns

Index(['SKU', 'Cód. Fabricante', 'Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Grupo de Produtos',
       'Grupo Indústria', 'Grupo de Meta', 'Grupo MCCAIN', 'Linha Fabrica',
       'Grupo de Promotores', 'Peso Liquido', 'Peso Bruto', 'Fator CX',
       'Unid. CMP', 'Unid. STK', 'Fator CMP', 'Fotos', 'EAN', 'DUN',
       'Peso Liquido Caixa', 'Peso Bruto Caixa', 'Fator Unid CMP'],
      dtype='object')

In [8]:
tabelas['Cliente'].columns

Index(['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'ID Segmento', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato', 'Limite de Crédito',
       'Principal', 'E-mail Cliente', 'Tributação do Cliente', 'ID Rota',
       'Latitude', 'Longitude'],
      dtype='object')

In [9]:
vendas_df=vendas_df.merge(tabelas['Produto'],on='SKU',how='inner')[['Origem', 'Situação Entrega', 'ID Situação', 'Situação', 'Pedido',
       'NFe', 'Tipo de Pedido', 'Tipo de Operação', 'Tabelas',
       'Data de Emissão', 'Data de Faturamento', 'Data de Entrega',
       'ID Empresa', 'ID Cliente', 'ID Vendedor', 'SKU','Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Grupo de Produtos',
       'Grupo Indústria', 'Seq', 'Qtde',
       'Unid. VDA', 'Fator', 'Qtde. VDA', 'Valor VDA', 'Total Venda',
       'Total AV', 'MG CRP', 'Margem CUE', 'Margem CMP', 'Comsissão R$',
       'COFINS R$', 'PIS R$', 'ICMS R$', 'ICMS ST R$', 'IPI R$',
       'Peso Bruto KG', 'Peso Líquido KG', 'Total Geral']]

In [10]:
grp_df=pd.DataFrame()

grp_df=vendas_df[['ID Cliente','Fabricante','Total Venda']].groupby(['ID Cliente','Fabricante'],as_index=False).sum()

In [11]:
codigos=grp_df['ID Cliente'].loc[grp_df['Fabricante'].isin(['FRAGOLE','DE MARCHI'])].unique().tolist()

In [12]:
id_clientes=grp_df['ID Cliente'].loc[~grp_df['ID Cliente'].isin(codigos)].unique().tolist()

In [13]:
cli_df=pd.DataFrame()

cli_df=vendas_df[['ID Cliente','Total Venda']].loc[vendas_df['ID Cliente'].isin(id_clientes)].groupby(['ID Cliente'],as_index=False).sum()

In [14]:
cli_df.columns

Index(['ID Cliente', 'Total Venda'], dtype='object')

In [15]:
cli_df=cli_df.merge(tabelas['Cliente'],on='ID Cliente',how='inner')[['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'ID Segmento', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato', 'Limite de Crédito',
       'Principal', 'E-mail Cliente', 'Tributação do Cliente', 'ID Rota',
       'Latitude', 'Longitude','Total Venda']]

In [16]:
cli_df=cli_df.loc[(cli_df['Tipo de Cliente']=='J')&(cli_df['Status do Cliente']=='ATIVO')]

In [17]:
marcas_df=pd.DataFrame()

marcas_df=grp_df.pivot(index='ID Cliente',columns='Fabricante',values='Total Venda').reset_index()

In [18]:
cli_df=cli_df.merge(marcas_df,on='ID Cliente',how='inner')

In [22]:
tabelas['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro', 'Status do Vendedor'],
      dtype='object')

In [23]:
tabelas['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente'],
      dtype='object')

In [27]:
tabelas['Vendedor']=tabelas['Vendedor'].merge(tabelas['Supervisor'],on='ID Equipe',how='inner')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria','Status do Vendedor','Supervisor', 'Email Sup','Gerente', 'Email Gerente']]

In [28]:
tabelas['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Status do Vendedor', 'Supervisor', 'Email Sup', 'Gerente',
       'Email Gerente'],
      dtype='object')

In [30]:
cli_df.columns

Index(['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'ID Segmento', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato', 'Limite de Crédito',
       'Principal', 'E-mail Cliente', 'Tributação do Cliente', 'ID Rota',
       'Latitude', 'Longitude', 'Total Venda', 'ABAIXO DE ZERO', 'ALFAMA',
       'AVIKO', 'BEM BRASIL', 'BGF', 'BIMBO', 'BONDUELLE', 'BUONO GEL',
       'CONTE', 'CUNHA', 'DAUCY', 'DE MARCHI', 'DELAMARIE', 'DI BASI',
       'DOLCE FAMIGLIA PJ ALIM', 'DPA', 'FEINKOST', 'FRAGOLE', 'FRESKIMASSAS',
       'FRUTA BRASIL', 'GRANO', 'HAVITA', 'HL INDUSTRIAL DE ALIMENTOS ',
       'LAMBWESTON', 'LUZA', 'MATHEUS', 'MAWE', 'MCCAIN', 'NESTLE', 'PADRÃO',
       'PASTA NOBRE', 'PRACTICE', 'R

In [34]:
cli_df=cli_df.merge(tabelas['Vendedor'],left_on='Principal',right_on='ID Vendedor',how='inner')

In [39]:
cli_df.loc[~cli_df['Equipe'].str.contains('120')].to_excel('Clientes.xlsx',index=False,encoding='ISO-8859-1')

In [40]:
cli_df.loc[cli_df['Equipe'].str.contains('120')]

,ID Cliente,CNPJ,CNPJ Caracter,Razão Social,Nome Fantasia,Tipo de Cliente,Status do Cliente,ID Segmento,Matriz,Crédito,Data de Cadastro,Primeira Compra,Última Compra,Dias Compra,Tabela,Condição de Pagto,Prazo Pagto,Pagamento,CEP,Endereço,Bairro,Município,Numero,Complemento,Região,UF,DDD,Contato,Limite de Crédito,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Status do Vendedor,Supervisor,Email Sup,Gerente,Email Gerente,E-mail Cliente,Tributação do Cliente,ID Rota,Latitude,Longitude,Total Venda,ABAIXO DE ZERO,ALFAMA,AVIKO,BEM BRASIL,BGF,BIMBO,BONDUELLE,BUONO GEL,CONTE,CUNHA,DAUCY,DE MARCHI,DELAMARIE,DI BASI,DOLCE FAMIGLIA PJ ALIM,DPA,FEINKOST,FRAGOLE,FRESKIMASSAS,FRUTA BRASIL,GRANO,HAVITA,HL INDUSTRIAL DE ALIMENTOS,LAMBWESTON,LUZA,MATHEUS,MAWE,MCCAIN,NESTLE,PADRÃO,PASTA NOBRE,PRACTICE,REGILU,SABOR E SABOR,SEARA,SITIO SÃO JOÃO,TOP AÇAI,TRIGUERIA
27,2990,08338987000160,14,JB2 RESTAURANTE COMIDA TIPICA LTDA - EPP,DIVINO FOGAO - CENT,J,ATIVO,0004,DIVINO FOGÃO,NORMAL,2006-11-17,2006-11-17,2019-10-02,1097,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,3153001.0,AV: DR. FRANCISCO MESQUITA,QUINTA DA PAINEIRA,SÃO PAULO,1000,None,MOOCA,SP,11,6.163352e+07,NaN,CLISEMCO,CLIENTE 120,CLIENTE 120,EQUIPE 12 (120),,AVULSO,ATIVO,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,PAULO.CAMELO@DIVINOFOGAO.COM.BR,PESSOA JURIDICA - CONTRI SIMPLES,ANALIA,-23.593356,-46.584643,1053.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,957.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,3226,06014605000118,14,RGB NORTE RESTAURANTE LTDA - EPP,DIVINO FOGAO - CENT,J,ATIVO,0004,DIVINO FOGÃO,NORMAL,2007-03-05,2007-03-05,2019-10-22,1077,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,2047050.0,TRAVESSA CASALBUONO,VILA GUILHERME,SÃO PAULO,120,None,SANTANA,SP,11,2.252257e+07,NaN,CLISEMCO,CLIENTE 120,CLIENTE 120,EQUIPE 12 (120),,AVULSO,ATIVO,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,PAULO.CAMELO@DIVINOFOGAO.COM.BR,PESSOA JURIDICA - CONTRI SIMPLES,Z.NORTE,-23.517471,-46.617031,4087.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3777.27,NaN,NaN,NaN,NaN,200.20,NaN,NaN,NaN,NaN,NaN
29,3474,08346961000164,14,BURGUER PLACE LANCHONETE E FAST FOOD LTD,BURGUER PLACE,J,ATIVO,0007,None,NORMAL,2007-05-28,2007-05-28,2019-04-12,1270,TABELA DE PREÇO,BOLETO BANCARIO 21 DIAS,21.0,BANCO,4026002.0,RUA DR. BACELAR,VILA CLEMENTINO,SÃO PAULO,926,None,MOEMA,SP,0,1.111111e+10,NaN,CLISEMCO,CLIENTE 120,CLIENTE 120,EQUIPE 12 (120),,AVULSO,ATIVO,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,,PESSOA JURIDICA - CONTRI SIMPLES,SAUDE,-23.603524,-46.648535,3256.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3147.29,NaN,NaN,NaN,NaN,109.62,NaN,NaN,NaN,NaN,NaN
30,3475,58019258000110,14,A LENHADORA CHURRASCARIA E REST LT-ME,A LENHADORA CHURRASC,J,ATIVO,0004,None,NORMAL,2007-05-28,2007-05-28,2019-04-23,1259,TABELA DE PREÇO,A VISTA,1.0,DINHEIRO,4144020.0,RUA FIACAO DA SAUDE,VILA DA SAÚDE,SÃO PAULO,14,None,SAUDE,SP,0,0.000000e+00,NaN,CLISEMCO,CLIENTE 120,CLIENTE 120,EQUIPE 12 (120),,AVULSO,ATIVO,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,None,PESSOA JURIDICA - CONTRI SIMPLES,SAUDE,-23.617626,-46.637660,2001.30,NaN,NaN,NaN,254.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1747.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,3487,61820346000120,14,PANIFICADORA SAO FERNANDO LTDA EPP,PANIF SAO FERNANDO,J,ATIVO,0017,None,NORMAL,2007-05-28,2007-05-28,2020-08-17,777,NESTLE PEQUENO VAREJO,BOLETO BANCARIO 07 DIAS,7.0,BANCO,4153001.0,RUA FRANCISCO TAPAJOS,VILA SANTO ESTÉFANO,SÃO PAULO,471,None,SAUDE,SP,0,0.000000e+00,